In [27]:
import mmap
import numpy as np
import binascii
import hashlib
import base58
import struct
import datetime
import hashlib

In [28]:
def publicKeyDecode(pub):
    if pub.lower().startswith('76a914'):
        pub = pub[6:-4]
        result = (b'\x00') + binascii.unhexlify(pub)
        h5 = hashlib.sha256(result)
        h6 = hashlib.sha256(h5.digest())
        result += h6.digest()[:4]
        return base58.b58encode(result)
    elif pub.lower().startswith('a9'):
        return ""
    elif startsWithOpNCode(pub):
        pub = pub[2:-2]
        h3 = hashlib.sha256(binascii.unhexlify(pub))
        h4 = hashlib.new('ripemd160', h3.digest())
        result = (b'\x00') + h4.digest()
        h5 = hashlib.sha256(result)
        h6 = hashlib.sha256(h5.digest())
        result += h6.digest()[:4]
        return base58.b58encode(result)
    return ""

In [29]:
def reverse(h):
    byte_array = bytearray(h)
    h_new = bytearray(b'0')
    for i in range(0,len(byte_array),2):
        h_new[i:i+1] = byte_array[len(byte_array)-1-i-1], byte_array[len(byte_array)-1-i]
        
    return bytes(h_new)

In [30]:
def hexToInt(value):
    return int(binascii.hexlify(value), 16)

def hexToStr(value):
    return binascii.hexlify(value)

def readVarInt(blockFile):
    varInt = ord(blockFile.read(1))
    returnInt = 0
    if varInt < 0xfd:
        return varInt
    if varInt == 0xfd:
          returnInt = readShortLittleEndian(blockFile)
    if varInt == 0xfe:
        returnInt = readIntLittleEndian(blockFile)
    if varInt == 0xff:
        returnInt = readLongLittleEndian(blockFile)
    return int(binascii.hexlify(returnInt), 16)

In [31]:
def stringLittleEndianToBigEndian(string):
    string = binascii.hexlify(string)
    n = len(string) / 2
    fmt = '%dh' % n
    return struct.pack(fmt, *reversed(struct.unpack(fmt, string)))

def readIntLittleEndian(blockFile):
    return struct.pack(">I", struct.unpack("<I", blockFile.read(4))[0])

def readShortLittleEndian(blockFile):
    return struct.pack(">H", struct.unpack("<H", blockFile.read(2))[0])

def readLongLittleEndian(blockFile):
    return struct.pack(">Q", struct.unpack("<Q", blockFile.read(8))[0])


In [152]:
def readInput(blockFile):
    previousHash = binascii.hexlify(blockFile.read(32)[::-1])
    outId = binascii.hexlify(blockFile.read(4))
    scriptLength = readVarInt(blockFile)
    scriptSignatureRaw = hexToStr(blockFile.read(scriptLength))
    scriptSignature = scriptSignatureRaw
    seqNo = binascii.hexlify(readIntLittleEndian(blockFile))

    print("\n" + "Input")
    print("-" * 20)
    print("> Previous Hash: ", previousHash)
    print("> Out ID: ", outId)
    print("> Script length: " + str(scriptLength))
    print("> Script Signature (PubKey) Raw: ", scriptSignatureRaw)
    print("> Script Signature (PubKey): ", scriptSignature)
    print("> Seq No: ", seqNo)

In [33]:
def readOutput(blockFile):
    value = hexToInt(readLongLittleEndian(blockFile)) / 100000000.0
    scriptLength = readVarInt(blockFile)
    scriptSignatureRaw = hexToStr(blockFile.read(scriptLength))
    scriptSignature = scriptSignatureRaw
    address = ''
    try:
        address = publicKeyDecode(scriptSignature)
    except:
        address = ''
  
    print("\n" + "Output")
    print("-" * 20)
    print("> Value: " + str(value))
    print("> Script length: " + str(scriptLength))
    print("> Script Signature (PubKey) Raw: ", scriptSignatureRaw)
    print("> Script Signature (PubKey): ", scriptSignature)
    print("> Address: " + address)

In [34]:
def readTransaction(blockFile):
    extendedFormat = False
    beginByte = blockFile.tell()
    inputIds = []
    outputIds = []
    version = hexToInt(readIntLittleEndian(blockFile)) 
    cutStart1 = blockFile.tell()
    cutEnd1 = 0
    inputCount = readVarInt(blockFile)
    
    print("\n\n" + "Transaction")
    print("-" * 100)
    print("Version: " + str(version))
    
    print("\nInput Count: " + str(inputCount))
    for inputIndex in range(0, inputCount):
        inputIds.append(readInput(blockFile))
    outputCount = readVarInt(blockFile)
    print("\nOutput Count: " + str(outputCount))
    for outputIndex in range(0, outputCount):
        outputIds.append(readOutput(blockFile))
        
    lockTime = hexToInt(readIntLittleEndian(blockFile))
    if lockTime < 500000000:
        print("\nLock Time is Block Height: " + str(lockTime))
    else:
        print("\nLock Time is Timestamp: " + datetime.datetime.fromtimestamp(lockTime).strftime('%d.%m.%Y %H:%M'))


In [139]:
def process_block(blockFile):
    magicNumber = binascii.hexlify(blockFile.read(4))
    blockSize = hexToInt(readIntLittleEndian(blockFile))
    version = hexToInt(readIntLittleEndian(blockFile))
    previousHash = reverse(binascii.hexlify(blockFile.read(32))
    merkleHash = reverse(binascii.hexlify(blockFile.read(32)))
    creationTimeTimestamp = hexToInt(readIntLittleEndian(blockFile))
    creationTime = datetime.datetime.fromtimestamp(creationTimeTimestamp).strftime('%d.%m.%Y %H:%M')
    bits = hexToInt(readIntLittleEndian(blockFile))
    nonce = hexToInt(readIntLittleEndian(blockFile))
    countOfTransactions = readVarInt(blockFile)
    
    print("Magic Number: ", magicNumber)
    print("Blocksize: " + str(blockSize))
    print("Version: " + str(version))
    print("Previous Hash: ", previousHash)
    print("Merkle Hash: ", merkleHash)
    print("Time: " , creationTime)
    print("Bits: " + str(bits))
    print("Nonce: " + str(nonce))
    print("Count of Transactions: " + str(countOfTransactions))
    
    for transactionIndex in range(0, countOfTransactions):
        readTransaction(blockFile)

In [153]:
i = 0
with open('blocks/blk00002.dat', 'rb') as file:
    while i < 10:
        print(i)
        process_block(file)
        print("")
        print("+"*100)
        print("")
        i+=1

0
Magic Number:  b'f9beb4d9'
Blocksize: 49069
Version: 1
Previous Hash:  b'0000000000000f054b58fa49589c03a98ff2fa5d6173a6b3c808230e5cfd8dea'
Merkle Hash:  b'e58c2d15327741e40bdc7e1e897a5805f9a1b85237fa07bf91fd7f55fe865922'
Time:  16.06.2011 13:33
Bits: 437461381
Nonce: 3528307584
Count of Transactions: 138


Transaction
----------------------------------------------------------------------------------------------------
Version: 1

Input Count: 1

Input
--------------------
> Previous Hash:  b'0000000000000000000000000000000000000000000000000000000000000000'
> Out ID:  b'ffffffff'
> Script length: 8
> Script Signature (PubKey) Raw:  b'048521131a022e04'
> Script Signature (PubKey):  b'048521131a022e04'
> Seq No:  b'ffffffff'

Output Count: 1

Output
--------------------
> Value: 50.2665
> Script length: 67
> Script Signature (PubKey) Raw:  b'410493051e31d731d8e37f92646c8275084b8a284ce438407215f80098fb790abe567558c058c1c075d5d5dd357b41f5d1a52e3d5406250ff37b0d2d544e4a0cfc2bac'
> Script Sig

> Script Signature (PubKey):  b'76a914af19bd1cd2da4a909ab430c599c54fd454c51bd988ac'
> Address: 

Lock Time is Block Height: 0


Transaction
----------------------------------------------------------------------------------------------------
Version: 1

Input Count: 2

Input
--------------------
> Previous Hash:  b'1392072a999133c9b1a7f1e6403e60ff21ee0deb03d1cfd5db21e9671693de09'
> Out ID:  b'01000000'
> Script length: 138
> Script Signature (PubKey) Raw:  b'473044022075f3bff1a1a07c6ffe2928621e33c0f273e0ed3bc53d0e416bb2479b27d64b7b0220672a6aaaa0b080057c8b1ce31d135965da14fbf289800372d1e411751fcda8d70141044f36955d8f8c8bc9387312b7c744c17a43e98c8b7caa940de49d1af1a4e70d230c48d44c899f1771a378c20d9f76cbae74720ee54236d5f53edf37f1150151fd'
> Script Signature (PubKey):  b'473044022075f3bff1a1a07c6ffe2928621e33c0f273e0ed3bc53d0e416bb2479b27d64b7b0220672a6aaaa0b080057c8b1ce31d135965da14fbf289800372d1e411751fcda8d70141044f36955d8f8c8bc9387312b7c744c17a43e98c8b7caa940de49d1af1a4e70d230c48d44c899f1771


Input Count: 1

Input
--------------------
> Previous Hash:  b'32eaf275d66754bd0e003f0ceaf03196fb420aa97d602c53f3d0e230ef0cfebb'
> Out ID:  b'00000000'
> Script length: 139
> Script Signature (PubKey) Raw:  b'483045022072f63f243dc04ee2b6f4c500d3aa9e6eec7660f7a6dc2abb5a6db28d6f121fe4022100dc69969f0e86fdeadf715db43fa92b7795e0fb0792719d80bf37495699dd4c09014104406a81b2a84770cea69ac7bca5c284fe67bdfd20d4d85bf7018482c02c4602f1b7b5ca8e8d2f52e0c4bbabfbbf5fba437f8a897e620303ea6cfbbbdf2a90137f'
> Script Signature (PubKey):  b'483045022072f63f243dc04ee2b6f4c500d3aa9e6eec7660f7a6dc2abb5a6db28d6f121fe4022100dc69969f0e86fdeadf715db43fa92b7795e0fb0792719d80bf37495699dd4c09014104406a81b2a84770cea69ac7bca5c284fe67bdfd20d4d85bf7018482c02c4602f1b7b5ca8e8d2f52e0c4bbabfbbf5fba437f8a897e620303ea6cfbbbdf2a90137f'
> Seq No:  b'ffffffff'

Output Count: 2

Output
--------------------
> Value: 5520.7
> Script length: 25
> Script Signature (PubKey) Raw:  b'76a914109afc68a4ece464dbcfad30dc6d431de04da78b88ac'
> Scr

Input Count: 1

Input
--------------------
> Previous Hash:  b'a79c810e36d0d37a8b881259f2065938f0531910495373d18c629c2d24873bec'
> Out ID:  b'00000000'
> Script length: 138
> Script Signature (PubKey) Raw:  b'47304402202f8351497917c53746c9b737d77f5b34fd4ca8fac5082addce987699a597b7fd0220533e23c1fc9c37bf36b81d8c84bd3a02c27aeafad0d54e2c24552a51d531a63d0141041e6c6158519c5555866661d27f408c03a40fbb06a9f8f0d3d14ff981891b1ad416e3eb07f993dfd10a3182d2f2e661b475e8a25571e60a0b8237cabc10f847fc'
> Script Signature (PubKey):  b'47304402202f8351497917c53746c9b737d77f5b34fd4ca8fac5082addce987699a597b7fd0220533e23c1fc9c37bf36b81d8c84bd3a02c27aeafad0d54e2c24552a51d531a63d0141041e6c6158519c5555866661d27f408c03a40fbb06a9f8f0d3d14ff981891b1ad416e3eb07f993dfd10a3182d2f2e661b475e8a25571e60a0b8237cabc10f847fc'
> Seq No:  b'ffffffff'

Output Count: 2

Output
--------------------
> Value: 60.09875044
> Script length: 25
> Script Signature (PubKey) Raw:  b'76a9146ed6a52f144653006971211631bf92ed7fe44ccb88ac'
> Scr

----------------------------------------------------------------------------------------------------
Version: 1

Input Count: 13

Input
--------------------
> Previous Hash:  b'7577534c5846ee190f297ff3f8452c03aab2323300c7b1fa4d8c9bda775c6530'
> Out ID:  b'00000000'
> Script length: 139
> Script Signature (PubKey) Raw:  b'483045022100c717ad85998bd999bccfdd3f6a58b29acf2c16b3ed50840471c197e2040c0fbd02204ad48e90b35cd41100a810d8dab9db775db1b998e6c361dc0a46d3e395738f70014104f5f7b56f751a9297ec764c4045286d6d1025fb23ee9f0b84854604f21acc663c0cdbdf7027748a8988d20f9003cf8b4d560c3def45e0c817d8691e16a3ee3bd4'
> Script Signature (PubKey):  b'483045022100c717ad85998bd999bccfdd3f6a58b29acf2c16b3ed50840471c197e2040c0fbd02204ad48e90b35cd41100a810d8dab9db775db1b998e6c361dc0a46d3e395738f70014104f5f7b56f751a9297ec764c4045286d6d1025fb23ee9f0b84854604f21acc663c0cdbdf7027748a8988d20f9003cf8b4d560c3def45e0c817d8691e16a3ee3bd4'
> Seq No:  b'ffffffff'

Input
--------------------
> Previous Hash:  b'49df553472e8d9

...

> Previous Hash:  b'1c928db7abd57b7c1b65c8f005903d360100ca5165d03afbc4923e97b6756614'
> Out ID:  b'01000000'
> Script length: 139
> Script Signature (PubKey) Raw:  b'4830450220419aacbe364969488327d71bf48220ee2d2653e018ec7db8caa7eccb26f6acc4022100c303f1dd954b314230bde12c9e964c7e2b495415ee9fd9d6f524ce80913c378f014104fc3c7456608e44a1f39f623d22c2f904a916c53d8a545937c699ec423b42a7698d4794d0c016e9b92aef7f6c443a701bcd4f93846d8b2ec7b363fb3f2b24efcf'
> Script Signature (PubKey):  b'4830450220419aacbe364969488327d71bf48220ee2d2653e018ec7db8caa7eccb26f6acc4022100c303f1dd954b314230bde12c9e964c7e2b495415ee9fd9d6f524ce80913c378f014104fc3c7456608e44a1f39f623d22c2f904a916c53d8a545937c699ec423b42a7698d4794d0c016e9b92aef7f6c443a701bcd4f93846d8b2ec7b363fb3f2b24efcf'
> Seq No:  b'ffffffff'

Input
--------------------
> Previous Hash:  b'0c746d6626108d888996bfbd88c3c3edf2707c969f66c88c1305b753abb0bbeb'
> Out ID:  b'01000000'
> Script length: 140
> Script Signature (PubKey) Raw:  b'493046022100f89397578a96b

> Script Signature (PubKey) Raw:  b'48304502202585f608031a0a223300795b01b16c39eed7161935646dd3ca29a16efb0c3a810221009bb3598c9b645d2e5362265b5e1aefc83a93c1fb852ef8e32359b3531d812a5e014104397fd9b7f9f73bf48b46bc030bf46136f2563a4d36e579cb40a52a6780ca00f50907ccc9b1bdb411895c040ed13548a7d4695a9856c76a5a10aaa0b6a40226f1'
> Script Signature (PubKey):  b'48304502202585f608031a0a223300795b01b16c39eed7161935646dd3ca29a16efb0c3a810221009bb3598c9b645d2e5362265b5e1aefc83a93c1fb852ef8e32359b3531d812a5e014104397fd9b7f9f73bf48b46bc030bf46136f2563a4d36e579cb40a52a6780ca00f50907ccc9b1bdb411895c040ed13548a7d4695a9856c76a5a10aaa0b6a40226f1'
> Seq No:  b'ffffffff'

Output Count: 2

Output
--------------------
> Value: 4436.3865
> Script length: 25
> Script Signature (PubKey) Raw:  b'76a914759d9702c8e4190bd64df7a64aad8a995f138eb988ac'
> Script Signature (PubKey):  b'76a914759d9702c8e4190bd64df7a64aad8a995f138eb988ac'
> Address: 

Output
--------------------
> Value: 43.27
> Script length: 25
> Script Signatu

> Previous Hash:  b'1ae98ca259f5d14b644985ed323dde520f23003dc42ec7b64c0675ed91948ca0'
> Out ID:  b'01000000'
> Script length: 138
> Script Signature (PubKey) Raw:  b'47304402205acc822d3b5affd7252d298f8b2c90dc196662e31b93c6730d07c801abdb537d0220112d63af373d44d1d145706848fcea4026e1b2799f1f1a051235522cde6cb24601410468c85fd34cc3cea0a7814e54184f0f5efd5b2dbdd697e815234b4177ba646257bad3688201b30e2d254d198db0b8c141ff915e9aeb894d9a7c2cf1d7d4df36d6'
> Script Signature (PubKey):  b'47304402205acc822d3b5affd7252d298f8b2c90dc196662e31b93c6730d07c801abdb537d0220112d63af373d44d1d145706848fcea4026e1b2799f1f1a051235522cde6cb24601410468c85fd34cc3cea0a7814e54184f0f5efd5b2dbdd697e815234b4177ba646257bad3688201b30e2d254d198db0b8c141ff915e9aeb894d9a7c2cf1d7d4df36d6'
> Seq No:  b'ffffffff'

Input
--------------------
> Previous Hash:  b'571ab5ef888e48faa0f00d9044926e344814d8946f88bf314e4a3d8a24f391b4'
> Out ID:  b'01000000'
> Script length: 139
> Script Signature (PubKey) Raw:  b'483045022100a8cff4c24bf7e565b

> Value: 2.36
> Script length: 25
> Script Signature (PubKey) Raw:  b'76a914ad4d112becfc60e5f9e2440ca307bc2dd498326188ac'
> Script Signature (PubKey):  b'76a914ad4d112becfc60e5f9e2440ca307bc2dd498326188ac'
> Address: 

Lock Time is Block Height: 0


Transaction
----------------------------------------------------------------------------------------------------
Version: 1

Input Count: 1

Input
--------------------
> Previous Hash:  b'd0f3e73d86d660c6b3bee7f859fe09f47d328caf6357ba9085decd3de57fa6f9'
> Out ID:  b'00000000'
> Script length: 138
> Script Signature (PubKey) Raw:  b'473044022038b00d7baef20ed9d053a879b0d116784a631275d7569d770baaa5bdc7aa028002206eb0933e0843a59eb81e254bcfa43e84c6db533a1a8f4bb9dfd469950a0beb900141040798c8b94de217bc0c1c6d3e7399a34583529f8ce86642459758b3e49e70ba7c2dbb3798a75ba92ccb1ea44bcd3060d0f321eb7fd7cc20a455c62dd453639552'
> Script Signature (PubKey):  b'473044022038b00d7baef20ed9d053a879b0d116784a631275d7569d770baaa5bdc7aa028002206eb0933e0843a59eb81e254bcfa4

> Address: 

Output
--------------------
> Value: 0.06
> Script length: 25
> Script Signature (PubKey) Raw:  b'76a91413871e95fc97b3df4dffca7b2d69ee8bbc1a2a0388ac'
> Script Signature (PubKey):  b'76a91413871e95fc97b3df4dffca7b2d69ee8bbc1a2a0388ac'
> Address: 

Output
--------------------
> Value: 0.1
> Script length: 25
> Script Signature (PubKey) Raw:  b'76a91446cf0cff136270c07bf9ddcc606515374d75080b88ac'
> Script Signature (PubKey):  b'76a91446cf0cff136270c07bf9ddcc606515374d75080b88ac'
> Address: 

Output
--------------------
> Value: 0.03
> Script length: 25
> Script Signature (PubKey) Raw:  b'76a9147486ab3480e50ef47333ff4cfccaf36aca75aaf388ac'
> Script Signature (PubKey):  b'76a9147486ab3480e50ef47333ff4cfccaf36aca75aaf388ac'
> Address: 

Output
--------------------
> Value: 0.21
> Script length: 25
> Script Signature (PubKey) Raw:  b'76a914798527f695ec915d249b9f2b526699ab67f0586988ac'
> Script Signature (PubKey):  b'76a914798527f695ec915d249b9f2b526699ab67f0586988ac'
> Address: 



In [191]:
with open('blk00000.dat', 'rb') as file:
    file.read(293)
    print(file.tell())

293


In [ ]:
0437cd7f8525ceed2324359c2d0ba26006d92d856a9c20fa0241106ee5a597c9

In [154]:
reverse(b'9f1fdf6d28719531ef2262a277b083b91528ca58d3231e2fd25e980a3dd370ef')

b'ef70d33d0a985ed22f1e23d358ca2815b983b077a26222ef319571286ddf1f9f'

In [133]:
len('0000000001ec3a985ee53d32ef164b58f69ab50792d1f97119b4780a96d476d1')

64

In [44]:
int.from_bytes(binascii.unhexlify(h), byteorder='little')

1

In [41]:
reverse(b'01\x00\x00')

b'\x00\x0001'

In [92]:
def _byte(b):
    return bytes((b, ))

def encodeVarInt(h):
    n = int.from_bytes(binascii.unhexlify(h), byteorder='little')
    l = 0
    tmp = []
    data = ""

    while True:
        tmp.append(n & 0x7F)
        if l != 0:
            tmp[l] |= 0x80
        if n <= 0x7F:
            break
        n = (n >> 7) - 1
        l += 1

    tmp.reverse()
    for i in tmp:
        data += format(i, '02x')
    return data

In [126]:
l = (b'00100000')

In [131]:
encodeVarInt(l)

'9f00'

In [130]:
bytes(res, 'utf-8')

b'9f00'

In [74]:
int.from_bytes(binascii.unhexlify(res), byteorder='little')


Error: Non-hexadecimal digit found

In [155]:
reverse(b'00000016b525e4287aa867d1312572ea6f774910d553e58471c8f83a01b1d2c2')

b'c2d2b1013af8c87184e553d51049776fea722531d167a87a28e425b516000000'